In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

api_key = os.getenv('UPSTAGE_API_KEY')
file_path = "../assets/[첨부] 회사생활가이드_2023.05.pdf"

# For image files, set use_ocr to True to perform OCR inference on the document before layout detection.
loader = UpstageLayoutAnalysisLoader(file_path
                                     , split="page"
                                     , api_key=api_key
                                     , use_ocr=True)

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
pages = loader.load()  # or loader.lazy_load()
 
for page in pages:
    print(page) # print the document content

page_content='<h1 id='0' style='font-size:22px'>[ 회사생활 가이드 ]</h1> <table id='1' style='font-size:16px'><tr><td></td><td>No part of be circulated, quoted,</td></tr><tr><td></td><td>it may or reproduced for distribution outside Hanwha Investment & Securities Co., Ltd. without prior written approval from Hanwha Investment & Securities Co., Ltd Hanwha Investment & Securities Co., Ltd. All rights reserved.</td></tr></table> <br><h1 id='2' style='font-size:18px'>2023. 05 I 인재관리팀</h1> <br><h1 id='3' style='font-size:20px'>한화투자증권</h1>' metadata={'page': 1}
page_content='<h1 id='5' style='font-size:22px'>목 차</h1> <p id='6' data-category='paragraph' style='font-size:20px'>I. 인사제도 .3<br>II. 복지제도 10<br>III. 연수제도 29<br>IV. 컴플라이언스 35<br>V. 정보보호 42<br>VI. 재무(카드, 교통비, 경조비) 53<br>VII. 비용지출 가이드 57<br>VIII. 그룹 内 임직원 혜택 사항 69</p>' metadata={'page': 2}
page_content='<h1 id='10' style='font-size:18px'>I. 인사제도</h1>' metadata={'page': 3}
page_content='<p id='14' data-category='list' style='font-size:18px'>직급체

In [18]:
print(pages[11].page_content)

<table id='8' style='font-size:18px'><tr><td>구 분</td><td>상 세 내 용</td></tr><tr><td>신청 자격 (게시판 '주택자금대여지원지침' 사전확인 必)</td><td>· 1년 이상의 계속 고용을 조건으로 채용된 직원으로서 근속연수 1년 이상인자 (휴직기간은 근속연수 산정 시 제외), 또는 2년 이상의 경력 인정을 받아 채용된 직원 · 무주택 세대주 직원</td></tr><tr><td>주택 규모</td><td>· 국민주택규모 (85m2 이하)</td></tr><tr><td>대여 금액</td><td>· 5,000 만원 이내 (공동명의시 2,500만원 이내)</td></tr><tr><td>상환 방법</td><td>· 3년 거치 12년 균등분할상환 (중도 일시 상환 가능)</td></tr><tr><td>대여 기간</td><td>· 15년</td></tr><tr><td>이자율</td><td>가. 2천만원 이내: 1% 나. 2천만원 초과 4천만원 이내: 연 2% 다. 4천만원 초과분: 연 3%</td></tr><tr><td>신청 방법</td><td>· 구비서류 준비 ① 매매 또는 전세 계약서 사본 ② 주민등록등본 ③ 목적부동산의 등기사항전부증명서(등기부 등본) ④ 과세증명서 (세목별 과세증명서 본인 및 배우자 각 1부) ⑤ 신분증 사본, ⑥ 배우자가 직장이 있는 경우 주택자금 미 수혜 확인서 1부 : 당사 양식 준용 (무직인 경우는 전년도 건강보험자격사실 확인서 1부로 대체) ⑦ 재직증명서, ⑧ 최근 2년 원천징수영수증 각 1부 · 대출실행 1개월 전까지 구비서류를 인재관리팀 장인경 과장에게 송부 → 기본심사 후 신한은행 으로 전달 (이후 대출실행 과정은 신한은행에서 진행)</td></tr></table>


In [19]:
from bs4 import BeautifulSoup
from markdownify import markdownify as md

def html_to_markdown(html_content):
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 불필요한 요소 제거 (예: 스크립트, 스타일)
    for script in soup(["script", "style"]):
        script.decompose()
    
    # 특정 요소 전처리 (필요한 경우)
    # 예: 특정 클래스를 가진 div를 제거하거나 수정
    # for div in soup.find_all("div", class_="unwanted-class"):
    #     div.decompose()
    
    # markdownify를 사용하여 HTML을 Markdown으로 변환
    markdown = md(str(soup), heading_style="ATX", strip=['a'])
    
    return markdown.strip()


# pages 리스트를 markdown_pages로 복사
markdown_pages = pages

# 각 페이지의 page_content를 마크다운으로 변환하여 markdown_pages에 업데이트
for i, page in enumerate(pages):
    markdown_page = html_to_markdown(page.page_content)
    markdown_pages[i].page_content = markdown_page  # 변환된 내용을 page_content에 저장

In [22]:
# 출력
for i, page in enumerate(pages):
    print(f"{i+1} =================================================================")
    print(page)

1 =================================================================
page_content='# \[ 회사생활 가이드 ]

 

|  | No part of be circulated, quoted, |
| --- | --- |
|  | it may or reproduced for distribution outside Hanwha Investment \& Securities Co., Ltd. without prior written approval from Hanwha Investment \& Securities Co., Ltd Hanwha Investment \& Securities Co., Ltd. All rights reserved. |

   
# 2023\. 05 I 인재관리팀

   
# 한화투자증권' metadata={'page': 1}
2 =================================================================
page_content='# 목 차

 I. 인사제도 .3  
II. 복지제도 10  
III. 연수제도 29  
IV. 컴플라이언스 35  
V. 정보보호 42  
VI. 재무(카드, 교통비, 경조비) 53  
VII. 비용지출 가이드 57  
VIII. 그룹 内 임직원 혜택 사항 69' metadata={'page': 2}
3 =================================================================
page_content='# I. 인사제도' metadata={'page': 3}
4 =================================================================
page_content='직급체계 : 직무역할을 고려한 직급체계 → 종합직군(1갑\~3급), 사무지원직군(S4\~S1\)  
■ 호칭제도 : 직급에 따라 '부장 \- 차장 \- 과장 \- 대리 \- 주임